### Библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from tqdm import tqdm

import statsmodels.api as sm
from scipy.stats import t, f, boxcox, skew, kurtosis, gmean
from statsmodels.stats.diagnostic import linear_reset, het_white

import warnings
warnings.filterwarnings('ignore')

### Обработка

In [2]:
data = pd.read_csv('data_after_processing.csv', encoding='utf-8')

data = data.drop(['title'], axis=1)
data = data.drop(['author_Другой', 'publisher_Другой', 'publication_year_Другой',
                  'cover_type_Мягкий заламинированный картон', 'reading_age_6+'], axis=1)

### Логарифм цены

In [3]:
data['log_price'] = np.log(data['price'])
data = data.drop(['price'], axis=1)
y = data['log_price']
X = data.drop(['log_price'], axis=1)

### Регрессоры, которые можно логарифмировать и нет

In [4]:
cols_to_try_log = [
    'avg_rating', 'cnt_reviews', 'pages_cnt', 'tirage', 
    'weight', 'thickness', 'width', 'length', 'volume'
]

unconditional_cols = [x for x in X.columns.tolist() if x not in cols_to_try_log]

In [5]:
def generate_all_combinations(num_of_repeats):
    """Генерирует все комбинации признаков, к которым применяется функциональное преобразование"""
    return product([False, True], repeat=num_of_repeats)

def prepare_X(cols_to_try_func, combination, df, func):
    """Создает матрицу X для текущей комбинации"""
    """Примеры функций: np.log, np.power, np.reciprocal, ..."""
    X_temp = df[unconditional_cols].copy()
    feature_name = getattr(func, '__name__', repr(func))
    
    for i, col in enumerate(cols_to_try_func):
        if combination[i]:
            try:
                safe_col = df[col]
                if (safe_col <= 0).any() and (func == np.log) or (func == np.reciprocal):
                    safe_col = safe_col + 1e-6
                X_temp[f'{feature_name}_{col}'] = func(safe_col)
            except Exception as e:
                print(f"Ошибка при применении функции к {col}: {e}")
        else:
            X_temp[col] = df[col]
    
    return sm.add_constant(X_temp)

def getting_statistic(cols_to_try_func, df, func):
    results = []

    total_combinations = 2 ** len(cols_to_try_func)
    feature_name = getattr(func, '__name__', repr(func))

    for combination in tqdm(generate_all_combinations(num_of_repeats=len(cols_to_try_func)), total=total_combinations):
        try:
            # Подготавливаем данные
            X_curr = prepare_X(cols_to_try_func, combination, df, func)

            model = sm.OLS(df['log_price'], X_curr).fit()
            results.append({
                'combination': combination,
                'aic': model.aic,
                'bic': model.bic,
                'adj_r2': model.rsquared_adj,
            })

        except Exception as e:
            print(f"Error in combination {combination}: {str(e)}")
            continue

    df_results = pd.DataFrame(results)

    df_results[f'{feature_name}_columns'] = df_results['combination'].apply(
        lambda x: [cols_to_try_func[i] for i, my_func in enumerate(x) if my_func]
    )
    
    return df_results

In [6]:
# Проверяем логарифмирование
data_frame = getting_statistic(cols_to_try_func=cols_to_try_log, df=data, func=np.log)

best_aic = data_frame.loc[data_frame['aic'].idxmin()]
best_bic = data_frame.loc[data_frame['bic'].idxmin()]
best_adj_r2 = data_frame.loc[data_frame['adj_r2'].idxmax()]

print("Лучшая модель по AIC:")
print(f"Логарифмированные переменные: {best_aic['log_columns']}")
print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Логарифмированные переменные: {best_bic['log_columns']}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Логарифмированные переменные: {best_adj_r2['log_columns']}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")

100%|█████████████████████████████████████████| 512/512 [00:09<00:00, 52.00it/s]

Лучшая модель по AIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
AIC: -394.16

Лучшая модель по BIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
BIC: -181.07

Лучшая модель по Adj.R²:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
Adj.R²: 0.8077


In [7]:
# Проверяем признаки вида 1/x
my_func = np.reciprocal
data_frame = getting_statistic(cols_to_try_func=cols_to_try_log, df=data, func=my_func)

best_aic = data_frame.loc[data_frame['aic'].idxmin()]
best_bic = data_frame.loc[data_frame['bic'].idxmin()]
best_adj_r2 = data_frame.loc[data_frame['adj_r2'].idxmax()]

column_name = f'{my_func.__name__}_columns'

print("Лучшая модель по AIC:")
print(f"Преобразованные переменные: {best_aic[column_name]}")

print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Преобразованные переменные: {best_bic[column_name]}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Преобразованные переменные: {best_adj_r2[column_name]}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")

100%|█████████████████████████████████████████| 512/512 [00:09<00:00, 52.87it/s]

Лучшая модель по AIC:
Преобразованные переменные: []
AIC: -266.76

Лучшая модель по BIC:
Преобразованные переменные: []
BIC: -53.67

Лучшая модель по Adj.R²:
Преобразованные переменные: []
Adj.R²: 0.8000


Также функции будут работать, если мы решим проверить какие-то кастомные функции типо этого:

In [8]:
def quadratic_shift(x):
    return x**2 + 3*x + 5

### Итоговая модель с ln Y

После проверки всех функциональных форм обучаем итоговую модель:

In [9]:
cols_to_log = ['pages_cnt', 'tirage', 'weight', 'thickness']
cols_not_to_log = [col for col in cols_to_try_log if col not in cols_to_log] + unconditional_cols

X_log = np.log(data.loc[:, cols_to_log])
X_not_to_log = data.loc[:, cols_not_to_log]

X_log_model = sm.add_constant(pd.concat((X_log, X_not_to_log), axis=1))

log_model = sm.OLS(data['log_price'], X_log_model).fit()
log_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     403.0
Date:                Mon, 05 May 2025   Prob (F-statistic):               0.00
Time:                        17:54:41   Log-Likelihood:                 232.08
No. Observations:                3256   AIC:                            -394.2
Df Residuals:                    3221   BIC:                            -181.1
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               5.1408      0.285     18.012      0.000       4.581       5.700
pages_cnt                          -0.4177      0.022    -18.714      0.000      -0.461      -0.374
tirage                             -0.1707      0.010    -17.330      0.000      -0.190      -0.151
weight                              1.0142      0.022     46.985      0.000       0.972       1.057
thickness                          -0.4881      0.035    -14.044      0.000      -0.556      -0.420
avg_rating                          0.0484      0.012      4.132      0.000       0.025       0.071
cnt_reviews                         0.0003   3.46e-05      9.469      0.000       0.000       0.000
width                              -0.0394      0.003    -13.469      0.000      -0.045      -0.034
length                             -0.0164      0.004     -3.927      0.000      -0.025      -0.008
volume                              0.0006   3.63e-05     16.473      0.000       0.001       0.001
author_Джейн Остен                  0.0172      0.028      0.614      0.539      -0.038       0.072
author_Джек Лондон                 -0.0631      0.027     -2.300      0.022      -0.117      -0.009
author_Джордж Оруэлл                0.0110      0.029      0.381      0.704      -0.046       0.068
author_Лев Толстой                 -0.1110      0.031     -3.611      0.000      -0.171      -0.051
author_Луиза Мэй Олкотт            -0.0269      0.032     -0.833      0.405      -0.090       0.036
author_Михаил Булгаков              0.0758      0.027      2.856      0.004       0.024       0.128
author_Николай Гоголь              -0.0741      0.033     -2.266      0.024      -0.138      -0.010
author_Федор Достоевский           -0.0818      0.021     -3.914      0.000      -0.123      -0.041
author_Эрих Ремарк                  0.0881      0.029      3.027      0.002       0.031       0.145
publisher_АСТ                      -0.0882      0.030     -2.949      0.003      -0.147      -0.030
publisher_Азбука                   -0.0830      0.031     -2.676      0.007      -0.144      -0.022
publisher_Иностранка                0.0033      0.035      0.094      0.925      -0.065       0.072
publisher_Манн, Иванов и Фербер     0.0183      0.040      0.457      0.648      -0.060       0.097
publisher_Эксмо                    -0.1106      0.030     -3.654      0.000      -0.170      -0.051
publication_year_2021              -0.0390      0.045     -0.870      0.384      -0.127       0.049
publication_year_2022              -0.0056      0.038     -0.149      0.882      -0.079       0.068
publication_year_2023               0.0129      0.036      0.359      0.720      -0.058       0.084
publication_year_2024               0.0355      0.035      1.001      0.317      -0

**мини-вопрос на подумать:** допустим, мы попробовали разные функциональные формы для призаков и в качестве того, для чего резонно применять `log` и `custom_func` у нас функция выдала один и тот же признак `tirage`. Что с ним делать?
<br>
<br>
**"ответ":** я предлагаю сильно не заморачиваться и выбрать что придется